# Decision Trees

In [5]:
import numpy as np

* Nodes: decision nodes
* Edges: values
* Leafs: output / decision / answer
* Inductive inference
* Robust to noisy data
* Can learn disjunctive expressions
* Completely expressive hypothesis space
* Represent a disjunction of conjunctions
    * Each path to a leaf is a conjunction
    * The tree as a whole is a disjunction
* Handle missing values
* Variants: ID3, ASSISTANT, C4.5
* Greedy search (never backtracks)

### Inductive bias
- Restriction bias
    - Completely expressive
- Preference bias
    - Prefers good splits early on
    - Prefers correct trees
    - Prefers shorter trees (a consequence of the 1st preference)

### Entropy
* How homogenous/pure are the examples in a set
* If perfectly pure then entropy is 0 (if all is the same value)
* Entropy is maximum for a given cardinality (i.e. number of unique items in a set) when no item is repeated

$$-\sum_{v}p(v)logp(v)$$

In [2]:
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    p = counts / np.sum(counts)
    return -np.sum(p * np.log2(p))

def entropy_binary(x):
    trues = np.sum(x)
    percentage_true = trues / len(x)
    if percentage_true == 1:
        return 1
    else:
        percentage_false = 1 - percentage_true
        p = np.array([percentage_true, percentage_false])
        return -np.sum(p * np.log2(p))

In [72]:
x1 = [True, False, False, False]
x2 = [True, False]
x3 = [True, True, True]
x4 = [True] * 99 + [False]

In [73]:
entropy(x1)

0.81127812445913283

In [74]:
entropy(x2)

1.0

In [75]:
entropy(x4)

0.080793135895911181

In [76]:
entropy(x3)

-0.0

In [77]:
entropy_binary(x1)

0.81127812445913283

In [78]:
entropy_binary(x2)

1.0

In [79]:
entropy_binary(x3)

1

In [80]:
# TODO: not exactly the same
entropy_binary(x4)

0.080793135895911236

### Information gain
$$Gain(S, A)=Entropy(S)-\sum_{v}\frac{|S_v|}{|S|}Entropy(S_v)$$

In [108]:
test_cases = [
    ({'attribute_values': [True, True, False, False], 'ys': [True, True, False, False]}, 1.0),
    ({'attribute_values': [True, True, False, False], 'ys': [False, True, False, True]}, 0.0),
    ({'attribute_values': [False, True, False, False], 'ys': [False, True, False, True]}, 0.31127812445913283),
]

In [147]:
def information_gain_naive(attribute_values, ys):
    e = entropy(ys)
    
    a_stats = {True: [], False: []}
    for a, y in zip(attribute_values, ys):
        a_stats[a].append(y)
    
    a_entropy = sum([
        len(a_stats[val]) * entropy(a_stats[val])
        for val in a_stats
    ]) / len(ys)
    return e - a_entropy

def information_gain_vectorized(attribute_values, ys):
    attribute_values = np.array(attribute_values)
    ys = np.array(ys)
    e = entropy(ys)
    true_mask = attribute_values == True
    ys_true = ys[true_mask]
    ys_false = ys[~true_mask]
    a_entropy = (len(ys_true) * entropy(ys_true)
                 + len(ys_false) * entropy(ys_false)) / len(ys)
    return e - a_entropy

In [132]:
all([
    information_gain_naive(**inputs) == expected_output
    for inputs, expected_output
    in test_cases
])

True

In [135]:
all([
    information_gain_vectorized(**inputs) == expected_output
    for inputs, expected_output
    in test_cases
])

True

In [144]:
av = [True] * 10000 + [False] * 20000 + [True] * 30000
ys = [False] * 10000 + [True] * 20000 + [True] * 30000

In [148]:
%%timeit
information_gain_vectorized(av, ys)

100 loops, best of 3: 7.08 ms per loop


In [146]:
%%timeit
information_gain_naive(av, ys)

10 loops, best of 3: 25.9 ms per loop


### ID3: top-down algorithm for inducing a decision tree
- Loop:
    - Pick the best attribute A to split on
    - Assign A as decision node
    - Assign examples to leafs for each possible value of A
    - Stop if the stopping criterion is met

In [52]:
test_cases = [
    ({'xs': [{}, {}], 'ys': [True, True]}, {'root': True}),  # no attributes
    ({'xs': [{}, {}, {}], 'ys': [True, False, False]}, {'root': False}),
    ({'xs': [{'a': True}, {'a': False}], 'ys': [True, True]} , {'root': True}),  # all labels the same
    ({'xs': [{'a': True}, {'a': False}], 'ys': [False, False]} , {'root': False}),
    
]

In [151]:
class ID3:
    def __init__(self):
        self.nodes = {}
        self.current_key = 'root'
        self.attributes = None
        
    def _best_attribute(self, xs, ys):
        if self.attributes is None:
            # Assuming all rows have the same attributes (dense features)
            self.attributes = set(xs[0].keys())
        
        return max([
            (information_gain_vectorized([
                xs[attribute]
                for instance in xs
            ], ys), attribute)
            for attribute in self.attributes
        ], key=lambda x: x[0])[1]
        
    
    def train(self, xs, ys):
        unique_ys, unique_y_counts = np.unique(ys, return_counts=True)
        if all([len(x) == 0 for x in xs]):
            # No attributes
            self.nodes[self.current_key] = unique_ys[np.argmax(unique_y_counts)]
        else:
            if len(unique_ys) == 1:
                # All labels the same
                self.nodes[self.current_key] = unique_ys[0]
            else:
                best_attribute = self._best_attribute(xs, ys)
                print(best_attribute)
                
        return self.nodes

In [152]:
ID3().train([{'a': True, 'b': False},
             {'a': True, 'b': False},
             {'a': False, 'b': False},
             {'a': False, 'b': True}], [True, True, False, True])

TypeError: list indices must be integers or slices, not str

In [68]:
all([
    ID3().train(**inputs) == expected_output
    for inputs, expected_output
    in test_cases
])

True

### References
- https://www.udacity.com/course/machine-learning--ud262
- [Machine Learning by Tom Mitchell](https://www.amazon.com/Learning-McGraw-Hill-International-Editions-Computer/dp/0071154671/ref=asap_bc?ie=UTF8)
- https://blog.slavv.com/identifying-churn-drivers-with-random-forests-65bad0193e6b